# Trabajo de Fin de Grado: Aplicación de Inteligencia artificial a la predicción de los efectos de la radiación en sistemas digitales complejos

### Pablo Darós Pallarés

En este notebook se busca optimizar los modelos MLP. Para ello se prueban alternativas para reducir la dimensionalidad:
- SelectFromModel
- Principal Component Analysis (PCA)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import KFold, GridSearchCV, train_test_split, learning_curve
from sklearn.metrics import classification_report, make_scorer, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

##### MLP Classifier

SelectFromModel

In [ ]:
# Cargar los datos
df_one_hot = pd.read_csv('data/radecs_one_hot_label.csv', delimiter=",")
df_unique_label = pd.read_csv('data/radecs_unique_label.csv', delimiter=",")

# Funcion para comprobar si los valores de SEE son lists o str
def parse_see_value(value):
    if isinstance(value, str):
        try:
            return eval(value)
        except:
            return [value]  # Caso en que el eval no sea necesario
    elif isinstance(value, list):
        return value
    else:
        return [value]

# Se comprueba si SEE es un list o str
df_unique_label['SEE'] = df_unique_label['SEE'].apply(parse_see_value)

# Convertir SEE para un modelo multietiqueta
mlb = MultiLabelBinarizer()
see_encoded = mlb.fit_transform(df_unique_label['SEE'])

# Se añaden las nuevas columnas al df
see_encoded_df = pd.DataFrame(see_encoded, columns=mlb.classes_, index=df_unique_label.index)
df_unique_label = pd.concat([df_unique_label, see_encoded_df], axis=1)

# Eliminar la columna original
df_unique_label.drop('SEE', axis=1, inplace=True)

# Se convierten las columnas categóricas a numéricas
categorical_cols = df_unique_label.select_dtypes(include=['object']).columns
df_unique_label = pd.get_dummies(df_unique_label, columns=categorical_cols, drop_first=True)

# Se eliminan carácteres especiales
df_unique_label.columns = df_unique_label.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

# Asegurar que cada columna tiene un nombre unico
def make_unique(column_names):
    seen = {}
    for i, name in enumerate(column_names):
        if name in seen:
            seen[name] += 1
            new_name = f"{name}_{seen[name]}"
            while new_name in seen:
                seen[name] += 1
                new_name = f"{name}_{seen[name]}"
            column_names[i] = new_name
        else:
            seen[name] = 1
    return column_names

df_unique_label.columns = make_unique(list(df_unique_label.columns))

# Se formatean correctamente los nombres de las clases
class_names = [name.replace(':', '').replace(',', '') for name in mlb.classes_]

# Se eliminan estas 7 etiquetas debido a ser demasiado infrecuentes
removed_labels = ['SEUAlpha', 'SEUPAlpha', 'SEUPNAlpha', 'SEUe', 'SETLP', 'SEFIHPL', 'SEGRN']
class_names = [name for name in class_names if name not in removed_labels]

# Se crean X e Y
X = df_unique_label.drop(columns=class_names, axis=1)
y = df_unique_label[class_names].values

# Se buscan las características más relevantees con SelectFromModel
selected_features = np.zeros(X.shape[1], dtype=bool)

for i in range(y.shape[1]):
    selector = SelectFromModel(estimator=LogisticRegression(solver='liblinear'), threshold="median").fit(X, y[:, i])
    selected_features = np.logical_or(selected_features, selector.get_support())

X_new = X.loc[:, selected_features]

# Se divide en train y test
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# Se crea la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Se crea el modelo
mlp_model = MLPClassifier(max_iter=1000)

# Hiper parametros
mlp_params = {
    'estimator__hidden_layer_sizes': [(50,)],  # Menos configuraciones de capas
    'estimator__alpha': [0.0001]
}

# Configurar GridSearchCV
mlp_clf = OneVsRestClassifier(mlp_model)
mlp_grid_search = GridSearchCV(mlp_clf, mlp_params, scoring=make_scorer(f1_score, average='micro', zero_division=0), cv=kf, n_jobs=-1, verbose=1)

mlp_grid_search.fit(X_train, y_train)

# Encontrar el mejor resultado
mlp_best_model = mlp_grid_search.best_estimator_
mlp_y_pred = mlp_best_model.predict(X_test)
mlp_report = classification_report(y_test, mlp_y_pred, target_names=class_names, zero_division=0)

print("MLP Classifier")
print(f"Mejores hiperparámetros: {mlp_grid_search.best_params_}")
print(mlp_report)

##################
# Codigo para crear las curvas de aprendizaje
train_sizes, train_scores, test_scores = learning_curve(mlp_best_model, X_train, y_train, cv=5, n_jobs=-1)

# Se calcula la media y la desv. estándar
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Se generan los graficos
plt.figure()
plt.title("Curva de Aprendizaje - MLP Classifier")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")

# Se dibujan las rectas
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

plt.legend(loc="best")
plt.show()


##### PCA

In [ ]:
# Cargar los datos
df_one_hot = pd.read_csv('data/radecs_one_hot_label.csv', delimiter=",")
df_unique_label = pd.read_csv('data/radecs_unique_label.csv', delimiter=",")

# Funcion para comprobar si los valores de SEE son lists o str
def parse_see_value(value):
    if isinstance(value, str):
        try:
            return eval(value)
        except:
            return [value]  # Caso en que el eval no sea necesario
    elif isinstance(value, list):
        return value
    else:
        return [value]

# Se comprueba si SEE es un list o str
df_unique_label['SEE'] = df_unique_label['SEE'].apply(parse_see_value)

# Convertir SEE para un modelo multietiqueta
mlb = MultiLabelBinarizer()
see_encoded = mlb.fit_transform(df_unique_label['SEE'])

# Se añaden las nuevas columnas al df
see_encoded_df = pd.DataFrame(see_encoded, columns=mlb.classes_, index=df_unique_label.index)
df_unique_label = pd.concat([df_unique_label, see_encoded_df], axis=1)

# Eliminar la columna original
df_unique_label.drop('SEE', axis=1, inplace=True)

# Se convierten las columnas categóricas a numéricas
categorical_cols = df_unique_label.select_dtypes(include=['object']).columns
df_unique_label = pd.get_dummies(df_unique_label, columns=categorical_cols, drop_first=True)

# Se eliminan carácteres especiales
df_unique_label.columns = df_unique_label.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

# Asegurar que cada columna tiene un nombre unico
def make_unique(column_names):
    seen = {}
    for i, name in enumerate(column_names):
        if name in seen:
            seen[name] += 1
            new_name = f"{name}_{seen[name]}"
            while new_name in seen:
                seen[name] += 1
                new_name = f"{name}_{seen[name]}"
            column_names[i] = new_name
        else:
            seen[name] = 1
    return column_names

df_unique_label.columns = make_unique(list(df_unique_label.columns))

# Se formatean correctamente los nombres de las clases
class_names = [name.replace(':', '').replace(',', '') for name in mlb.classes_]

# Se eliminan estas 7 etiquetas debido a ser demasiado infrecuentes
removed_labels = ['SEUAlpha', 'SEUPAlpha', 'SEUPNAlpha', 'SEUe', 'SETLP', 'SEFIHPL', 'SEGRN']
class_names = [name for name in class_names if name not in removed_labels]

# Se crean X e Y
X = df_unique_label.drop(columns=class_names, axis=1)
y = df_unique_label[class_names].values

# Se escala X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplica PCA
pca = PCA(n_components=25)  # Ajusta n_components según lo que necesites
X_pca = pca.fit_transform(X_scaled)

# Se divide en train y test
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Se crea la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Se crea el modelo
mlp_model = MLPClassifier(max_iter=1000)

# Hiper parametros
mlp_params = {
    'estimator__hidden_layer_sizes': [(50,)],  # Menos configuraciones de capas
    'estimator__alpha': [0.0001]
}

# Configurar GridSearchCV
mlp_clf = OneVsRestClassifier(mlp_model)
mlp_grid_search = GridSearchCV(mlp_clf, mlp_params, scoring=make_scorer(f1_score, average='micro', zero_division=0), cv=kf, n_jobs=-1, verbose=1)

mlp_grid_search.fit(X_train, y_train)

# Encontrar el mejor resultado
mlp_best_model = mlp_grid_search.best_estimator_
mlp_y_pred = mlp_best_model.predict(X_test)
mlp_report = classification_report(y_test, mlp_y_pred, target_names=class_names, zero_division=0)

print("MLP Classifier con PCA")
print(f"Mejores hiperparámetros: {mlp_grid_search.best_params_}")
print(mlp_report)

##################
# Codigo para crear las curvas de aprendizaje
train_sizes, train_scores, test_scores = learning_curve(mlp_best_model, X_train, y_train, cv=5, n_jobs=-1)

# Se calcula la media y la desv. estándar
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Se generan los graficos
plt.figure()
plt.title("Curva de Aprendizaje - MLP Classifier")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")

# Se dibujan las rectas
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

plt.legend(loc="best")
plt.show()
